https://github.com/davidsandberg/facenet  

https://github.com/nyoki-mtl/keras-facenet  

https://machinelearningmastery.com/how-to-develop-a-face-recognition-system-using-facenet-in-keras-and-an-svm-classifier/  

In [1]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
from keras.models import load_model
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import GridSearchCV
from sklearn import metrics
from sklearn.svm import SVC
from sklearn.utils import shuffle
import seaborn as sns
import pandas as pd
%matplotlib inline
pd.options.display.max_rows = 100

Using TensorFlow backend.


In [2]:
# Load face encoding model - FaceNet.
facenet_model = load_model('facenet_keras_model.h5')
# Summarize input and output shape
print(facenet_model.inputs)
print(facenet_model.outputs)

[<tf.Tensor 'input_1:0' shape=(None, 160, 160, 3) dtype=float32>]
[<tf.Tensor 'Bottleneck_BatchNorm/cond/Identity:0' shape=(None, 128) dtype=float32>]


/Users/uldis/Documents/Programming/virtual-environments/face-recognition/lib/python3.7/site-packages/keras/engine/saving.py:341: UserWarning: No training configuration found in save file: the model was *not* compiled. Compile it manually.
  warnings.warn('No training configuration found in save file: '


In [3]:
def read_img(filepath, mode="RGB"):
    """
    Function to read image from file and convert it to RGB by default, or leaves BGR.
    Input: file path.
    Output: image as numpy array.
    """
    img = cv2.imread(filepath)
    if mode == "RGB":
        # Convert to RGB
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    return img

In [4]:
def resize_img(image, required_size=(160, 160)):
    """
    Function to resize image.
    Input: image as numpy array.
    Param required_size: required_size as tuple (x,x); default is 160x160.
    Output: image as numpy array.
    """
    img = cv2.resize(image, required_size, interpolation=cv2.INTER_AREA)
    return img

In [5]:
def prewhiten_img(x):
    """
    Funtion to subtract the mean and normalize the range of the pixel values of input images.
    Input: either one image or list of images as numpy array.
    Output: numpy array.
    """
    # For list of images.
    if x.ndim == 4:
        axis = (1, 2, 3)
        size = x[0].size
    # For single image.
    elif x.ndim == 3:
        axis = (0, 1, 2)
        size = x.size
    else:
        raise ValueError('Dimension should be 3 or 4')

    mean = np.mean(x, axis=axis, keepdims=True)
    std = np.std(x, axis=axis, keepdims=True)
    std_adj = np.maximum(std, 1.0 / np.sqrt(size))
    y = (x - mean) / std_adj
    return y

In [6]:
def l2_normalize(x, axis=-1, epsilon=1e-10):
    """
    L2 normalization function to normalize 128D face embedding.
    Input: numpy array.
    Output: numpy array.
    """
    output = x / np.sqrt(np.maximum(np.sum(np.square(x), axis=axis, keepdims=True), epsilon))
    return output

In [7]:
def get_embedding(face_image, model=facenet_model):
    """
    Given an image, return the 128-dimension face encoding.
    Input: image as numpy array of shape (160, 160, 3).
    Param model: face embedding model; built-in (default) is FaceNet Keras model.
    Output: numpy array of shape (1, 128)
    """
    # Resize image to 160x160 (default dimensions applicable for FaceNet model).
    img = resize_img(face_image)
    # Normalize face image.
    img = prewhiten_img(img)
    # Exapand dimensions from 3 to 4 as model expects list of samples (4D array).
    sample = np.expand_dims(img, axis=0)
    # Make prediction to get embedding
    pred = model.predict(sample)
    # Normalize 128D output.
    pred = l2_normalize(pred)
    return pred

In [8]:
def calculate_distance(face_encodings, face_to_compare):
    """
    Given a list of face encodings, compare them to a known face encoding and get a euclidean distance
    for each comparison face. The distance tells you how similar the faces are.
    Input face_encodings: List of face encodings to compare. Should be numpy array of shape (1, 128) or list of them.
    Input face_to_compare: A face encoding to compare against. Should be numpy array of shape (1, 128)
    """
    return np.linalg.norm(face_encodings - face_to_compare, axis=1)

In [8]:
# Function to load image path and label into df.
# Faces of one person should be placed in one subdirectory. Subdirectory should be named after the person.
def load_face_images(directory, df):
    i = 0
    for subdir in os.listdir(directory):
        path = directory + '/' + subdir + '/'
        if not os.path.isdir(path):
            continue
        for filename in os.listdir(path):
            if filename.endswith(".jpg") or filename.endswith(".jpeg"):
                df.loc[i, "label"] = subdir
                df.loc[i, "img_path"] = path + filename
                i += 1

## Calculate distance

In [13]:
img_path1 = "/Users/uldis/Documents/Programming/python-projects/face-recognition/examples/faces/workplace_1.jpg"
img_path2 = "/Users/uldis/Documents/Programming/python-projects/face-recognition/examples/faces/workplace_4.jpg"
emb1 = get_embedding(read_img(img_path1))
emb2 = get_embedding(read_img(img_path2))
calculate_distance(emb1, emb2)

array([1.4311442], dtype=float32)

## Create pandas DataFrame with faces - img path, label, encoding

In [9]:
# Create empty pandas DataFrame to store source data.
df_data = pd.DataFrame()

In [10]:
# Load face image path and corresponding label from source dir.
source_dir = ""
load_face_images(source_dir, df_data)

In [11]:
# Shuffle data.
df_data = shuffle(df_data)

In [ ]:
print(df_data.shape)
df_data.head()

In [ ]:
# Visualize the number of each person in you df.
plt.figure(figsize=(16, 6))
sns.countplot(df_data.label)

In [15]:
# Get face embeddings.
df_data["embedding"] = df_data.img_path.apply(lambda x: get_embedding(resize_img(read_img(x)))[0])

In [8]:
# Store data to pickle file for later use.
df_data.to_pickle("./faces-df.pkl")
# Load data from pickle file.
# df_data = pd.read_pickle("./faces-df.pkl")

## Train (SVM) classifier

In [18]:
# Prepare data for training
X = np.asarray(list(df_data.embedding))
# label encode targets
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(df_data["label"])

In [19]:
# Split dataset
s = 700
X_train = X[:s]
X_test = X[s:]
y_train = y[:s]
y_test = y[s:]

In [ ]:
# (Optionaly) Use GridSearchCV to search for best params.
estimator = SVC(probability=True)
params = {"C": [0.01, 0.03, 0.1, 0.3, 1, 3, 10], "kernel": ["linear", "poly", "rbf", "sigmoid"]}

clf = GridSearchCV(estimator, params, scoring='f1_micro', n_jobs=-1, cv=5, verbose=True)
clf.fit(X_train, y_train)
clf.best_params_, clf.best_score_

In [ ]:
# Train model (with best parameters) and make predictions
SupVecClas = SVC(C=3, kernel="linear", probability=True)
SupVecClas.fit(X_train, y_train)
# Prediction
y_pred = SupVecClas.predict(X_test)
SupVecClas.score(X_test, y_test)

In [218]:
# Probabilities predictions. Max probability can differ from prediction.
y_prob = SupVecClas.predict_proba(X_test)
best_class_indices = np.argmax(y_prob, axis=1)
best_class_probabilities = y_prob[np.arange(len(best_class_indices)), best_class_indices]

In [ ]:
# Classification report
print(metrics.classification_report(y, SupVecClas.predict(X), target_names = label_encoder.classes_.tolist()))

## Predict person from sample face.

In [21]:
def predict_person(sample_img):
    embedding = get_embedding(resize_img(read_img(sample_img)))
    prediction = SupVecClas.predict(embedding)
    prediction = label_encoder.inverse_transform(prediction)
    return prediction

In [ ]:
img_path = ""
print("Person in the image is {}.".format(predict_person(img_path)[0]))

## Classify and move new face images to relevant folder using trained classifier.

In [155]:
from shutil import copyfile

In [157]:
face_dir = ""
dest_dir = ""
for filename in os.listdir(face_dir):
    if filename.endswith(".jpg"):
        s = os.path.join(face_dir, filename)
        person = predict_person(s)[0]
        d = os.path.join(dest_dir, person, filename)
        copyfile(s, d)